# Database Operations

Data is not limited to just text files. Data can also be retrieved and manipulated within databases. We will explore some of the ways to connect Python to a database for data analysis.

## SQLite

SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. SQLite is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day. If it is not already installed, you can download it from: https://www.sqlite.org/download.html

SQLite is an easy to way to be able to quickly connect to a database without installing an enterprise relational database, such as SQL Server, Oracle, or MySQL

### Using SQLite

1. Download the *books.sql* file from ELMS and place it in the same folder as this notebook.
2. Open the Anaconda Commpand Prompt, Terminal, or shell.
3. Navigate to the folder that contains the SQL file and Jupyter Notebook. You can use the `cd` command to change directories.
4. Execute the following statement: `sqlite3 books.db < books.sql`

The database will now be loaded and ready for use.

The database contains the following tables:

##### Authors - Stores all author information
- *id* - The author's unique ID number (integer)
- *first* - The author's first name (string)
- *last* - The author's last name (string)

##### Titles - Stores all book information
- *isbn* - The book's ISBN - International Standard Book Number (string)
- *title* - The book's title (string)
- *edition* - The book's edition number (integer)
- *copyright* - The book's copyright year (string)

##### author_ISBN - Associates authors to books 
- *id* - An author's ID number (integer)
- *isbn* - The book's ISBN (integer)

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Create a connection to the SQLite database
connection = sqlite3.connect('books.db')

The `read_sql` function will execute a SQL query and return a `DataFrame` containing the query's results. To call the function use:
- a string representing the query to be executed
- a `Connection` object referring to the database
- (optionally), the `index_col` argument denoting which column(s) should be used as the DataFrame's row indices, provided as a list

In [3]:
# Get all author data available
pd.read_sql('SELECT * FROM authors', connection)

,id,first,last
0,1,Paul,Deitel
1,2,Harvey,Deitel
2,3,Abbey,Deitel
3,4,Dan,Quirk
4,5,Alexander,Wald
5,6,Robert,Smith


In [4]:
# Get all author data available, and set the index of the DataFrame
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Robert,Smith


### Reading Data Without a Library

It is possible to read data from the database without a library using a `Cursor` object. Best practice dictates the use of exception handling in case something is wrong with the database connection.

In [5]:
# Prepare Cursor object
cursor = connection.cursor()
sql = 'SELECT first, last FROM authors'

# Try to execute the following code
try:
    # Execute the SQL command
    cursor.execute(sql)

    #Fetch all the rows into a list of tuples
    results = cursor.fetchall()
    for each_row in results:
        fname = each_row[0]
        lname = each_row[1]
        
        # Now print fetched result
        print("The author is: %s %s" % (fname, lname))

# If there are any errors executing the above code, don't cause a runtime error
except:
    print("OOPS! Something is wrong with retrieving the data")

The author is: Paul Deitel
The author is: Harvey Deitel
The author is: Abbey Deitel
The author is: Dan Quirk
The author is: Alexander Wald
The author is: Robert Smith


Read operations can typically occur in one of a few ways:
* **fetchone**: Fetch the next row of a query result set. A result set is an object that is returned when a Cursor object is used to query a table.
* **fetchmany(size)**: Fetch the number of rows in the result set specified by the size argument. When called repeatedly, this method fetches the next set of rows.
* **fetchall**: Fetches all rows on the result set.

### Practice Problem

Create three data frames, with appropriate indexing, for the various tables in the database and show the first five observations.

In [6]:
pd.read_sql('SELECT * FROM Authors ORDER BY last', connection, index_col='id').head()


,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
6,Robert,Smith


In [7]:
pd.read_sql('SELECT * FROM titles', connection,index_col=['isbn']).head()

,title,edition,copyright
isbn,,,
0135404673,Intro to Python for CS and DS,1,2020
0132151006,Internet & WWW How to Program,5,2012
0134743350,Java How to Program,11,2018
0133976890,C How to Program,8,2016
0133406954,Visual Basic 2012 How to Program,6,2014


In [8]:
pd.read_sql('SELECT * FROM author_isbn', connection).head(5)


,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


## Review of SQL Keywords

| SQL Keyword | Description |
|-------------|-------------|
| WHERE | Criteria to *filter rows* for selection that determine the rows to be retrieved, deleted or updated| 
| GROUP BY | Criteria to *filter groups of rows*|
| ORDER BY | Criteria for ordering rows|
| INNER JOIN | Merge rows from multiple tables based on a common attribute |
| LEFT JOIN | Merge all rows from a *left* table with rows from a *right* table. When a match is not found, `NULL` is returned |
| RIGHT JOIN | (Not supported in sqlite) Merge all rows from a *right* table with rows from a *left* table. When a match is not found from the *right* table, `NULL` is returned |
| OUTER JOIN | (Not supported in sqlite) Merge all rows from a *right* table with rows from a *left* table. When a match is not found from the opposite table, `NULL` is returned |
| INSERT INTO | Insert rows into a specified table |
| UPDATE | Update rows in a specified table |
| DELETE FROM | Delete rows from a specified table |

## WHERE Clause

You'll often select rows in a database that satisfy selection criteria.

In [9]:
# Select the books with a copyright later than 2016
pd.read_sql("""SELECT title, edition, copyright 
                FROM titles 
                WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


## Comparison, Equality, and Pattern Matching

The `WHERE` clause can contain comparison and equality operationrs, such as `<`, `<=`, `>`, `>=`, `=`, `<>` (not equal). It can also contain logical operators, such as `AND`, `OR`, and `NOT`.

The `LIKE` operator is used for pattern matching - searching for strings based on a given pattern.
* `_` to match exactly one character
* `%` to match 0 or more characters (wildcard)

In [10]:
# Pattern Matching - Zero or More Characters

pd.read_sql("""SELECT id, first, last 
                FROM authors 
                WHERE last LIKE 'D%'""", 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


In [11]:
# Pattern Matching - Any Character

pd.read_sql("""SELECT id, first, last 
                FROM authors 
                WHERE first LIKE '_b%'""", 
             connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel


## ORDER BY Clause

A query's results can be sorted in ascending (`ASC`) or descending (`DESC`) order. The default is ascending order

In [12]:
# Select all book titles ordered by title in ascending order
pd.read_sql('SELECT title FROM titles ORDER BY title ASC',
             connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


In [13]:
# Select all book titles ordered by title in descending order
pd.read_sql('SELECT title FROM titles ORDER BY title DESC',
             connection)

,title
0,Visual C++ How to Program
1,Visual C# How to Program
2,Visual Basic 2012 How to Program
3,Java How to Program
4,Intro to Python for CS and DS
5,Internet & WWW How to Program
6,C++ How to Program
7,C How to Program
8,Android How to Program
9,Android 6 for Programmers


To sort by multiple columns specify a comma-separated list of column names after the `ORDER BY` clause.

In [14]:
# Select all authors ordered first by last name and then by first name
pd.read_sql("""SELECT id, first, last 
                FROM authors 
                ORDER BY last, first""", 
             connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
6,Robert,Smith
5,Alexander,Wald


In [15]:
# Select all authors ordered first by last name in descending order and then by first name in descending order
pd.read_sql("""SELECT id, first, last 
                FROM authors 
                ORDER BY last DESC, first ASC""", 
             connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
6,Robert,Smith
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


## Combining the WHERE and ORDER BY Clauses

The `WHERE` and `ORDER BY` clauses can be combined in one query

In [16]:
# Get the isbn, title, edition, and copyright of each book that has a title ending with 'How to Program', sorting the results in ascending order by title
pd.read_sql("""SELECT isbn, title, edition, copyright
                FROM titles
                WHERE title LIKE '%How to Program'
                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


## Merging Data from Multiple Tables

Data can be merged from multiple tables, referred to as joining the tables. How the data is joined is dependent on the type of join.

In [17]:
# Select all authors and the books they have written
pd.read_sql("""SELECT first, last, isbn
                FROM authors
                INNER JOIN author_ISBN
                    ON authors.id = author_ISBN.id
                ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


In [18]:
# Select all authors regardless if they have written a book
pd.read_sql("""SELECT authors.id, first, last, isbn
                FROM authors
                LEFT JOIN author_ISBN
                    ON authors.id = author_ISBN.id
                ORDER BY last, first""", connection)

,id,first,last,isbn
0,3,Abbey,Deitel,0132151006
1,3,Abbey,Deitel,0133406954
2,2,Harvey,Deitel,0132151006
3,2,Harvey,Deitel,0133406954
4,2,Harvey,Deitel,0133976890
5,2,Harvey,Deitel,0134289366
6,2,Harvey,Deitel,0134444302
7,2,Harvey,Deitel,0134448235
8,2,Harvey,Deitel,0134601548
9,2,Harvey,Deitel,0134743350


### Practice Problems

What are the top 3 books with the most editions?

In [19]:
pd.read_sql("""SELECT *
                FROM titles
                ORDER BY edition DESC""", connection).head(3)

,isbn,title,edition,copyright
0,0134743350,Java How to Program,11,2018
1,0134448235,C++ How to Program,10,2017
2,0133976890,C How to Program,8,2016


What book titles have been written by authors with a first name starting with A?

In [20]:
pd.read_sql("""SELECT a.first, a.last, t.title
               FROM authors a
               INNER JOIN author_ISBN i
                   ON a.id = i.id
               INNER JOIN titles t
                   ON i.isbn = t.isbn
               WHERE a.first LIKE 'A%'""", connection)

,first,last,title
0,Alexander,Wald,Android 6 for Programmers
1,Abbey,Deitel,Internet & WWW How to Program
2,Abbey,Deitel,Visual Basic 2012 How to Program


Which books do not teach how to program?

In [21]:
pd.read_sql("""SELECT isbn, title, edition, copyright
               FROM titles
               WHERE title NOT LIKE '%How to Program%'
                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134289366,Android 6 for Programmers,3,2016
1,0135404673,Intro to Python for CS and DS,1,2020


## Data Modification

To modify data in the database, a `Cursor` object is needed.

In [ ]:
# Create a sqlite3 Cursor object
cursor = connection.cursor()

With a created cursor, statements that modify the database can be executed through the `execute` method.

### INSERT INTO Statements

In [22]:
# Add a new record to the authors table
cursor = cursor.execute("""INSERT INTO authors (first, last)
                            VALUES ('Sue', 'Red')""")

In [23]:
# Prove the new record was created
pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Robert,Smith
7,Sue,Red


In [24]:
# Be careful when the value has quotes! Double up in this case.
cursor = cursor.execute("""INSERT INTO authors (first, last)
                            VALUES ('Joe', 'O''Malley')""")
pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Robert,Smith
7,Sue,Red
8,Joe,O'Malley


After a modification is attempted, the `rowcount` attribute of the Cursor object will provide the number of records that were impacted.

In [25]:
# How many rows were inserted?
cursor.rowcount

1

### UPDATE Statements

In [26]:
# Change Sue Red's last name to Black
cursor = cursor.execute("""UPDATE authors SET last='Black'
                            WHERE last='Red' AND first='Sue'""") 

pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Robert,Smith
7,Sue,Black
8,Joe,O'Malley


In [27]:
# How many records were changed?
cursor.rowcount

1

### DELETE FROM Statements

In [28]:
# Delete the user whose name is Robert Smith or ID is 6
cursor = cursor.execute("""DELETE FROM authors WHERE id = 6 OR first='Robert' AND last = 'Smith'""") 

In [29]:
pd.read_sql('SELECT id, first, last FROM authors', 
             connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
7,Sue,Black
8,Joe,O'Malley


### Closing the Database Connection

When you are finished with the database, it is important to call the Connection's `close` method to disconnect from the database. The consequence of doing this is the database tables accessed will be locked until the connection is released.

In [ ]:
connection.close()

## MySQL

MySQL, an enterprise relational database, works very well with Python and Pandas. Much of the previous code will work in the same way for MySQL as it does with SQLite.

To use Python with MySQL:

1. Install MySQL
    * On Microsoft Windows - https://dev.mysql.com/doc/refman/8.0/en/windows-installation.html
    * On macOS - https://dev.mysql.com/doc/refman/8.0/en/osx-installation.html
    * On all other operating systems - https://dev.mysql.com/doc/refman/8.0/en/installing.html


2. Install MySQL Connector Python - https://pynative.com/install-mysql-connector-python/
    * At the Anaconda terminal, run the command: `pip install mysql-connector-python`
        * Note: Make sure you have run Anaconda as an administrator

### Side Note: Installing Other Python Packages

There are two primary ways additional packages get installed into a Python environment:

1. pip - Recommended tool for installing packages from the Python Package Index, PyPI (https://pypi.org/)
2. conda - Recommended tool for installing packages built in languages other than Python (e.g. C, C++, R, etc.)

Whenever you find a new library, the instructions will usually tell you if you should install the package via pip or conda.

For more details on pip vs. conda: https://www.anaconda.com/understanding-conda-and-pip/

### Using MySQL Connector Python

Download the *books-MySQL.sql* file from ELMS. Execute the script using MySQL Workbench.

In [30]:
# import the package
import mysql.connector

In [31]:
# Connect to the database
conn = mysql.connector.connect(user='python', password='python',
                              host='127.0.0.1',
                              database='python_data_analysis')

In [32]:
pd.read_sql('SELECT * FROM authors', conn)

,id,first,last
0,1,Paul,Deitel
1,2,Harvey,Deitel
2,3,Abbey,Deitel
3,4,Dan,Quirk
4,5,Alexander,Wald
5,6,Robert,Smith


Pandas is not required. A cursor could be used to query the database

In [33]:
mcursor = conn.cursor()
mcursor.execute("SELECT VERSION()")
results = mcursor.fetchall()
for each_row in results:
    print(each_row)

('8.0.16',)


In [34]:
mcursor = conn.cursor()

try:
    mcursor.execute('SELECT first, last FROM authors')
    results = mcursor.fetchall()
    for each_row in results:
        fname = each_row[0]
        lname = each_row[1]
        print('The author is: %s %s' % (fname, lname))
    print('\nNumber of records returned: %d' % (mcursor.rowcount))

except:
    print("OOPS! Something is wrong with retrieving the data")

The author is: Paul Deitel
The author is: Harvey Deitel
The author is: Abbey Deitel
The author is: Dan Quirk
The author is: Alexander Wald
The author is: Robert Smith

Number of records returned: 6


In [ ]:
# Don't forget to close the connection when done.
conn.close()